In [2]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD

import pandas as pd
import numpy as np

Using TensorFlow backend.
/home/dealer/miniconda3/envs/ai2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dealer/miniconda3/envs/ai2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dealer/miniconda3/envs/ai2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dealer/miniconda3/en

In [ ]:
lst = [[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]

df = pd.read_csv("data.csv")
data = np.array(df)

x = data[:, :6700]
y = data[:, 6700]

x = x.reshape(2969, 6700)
y = np.array([lst[int(i)] for i in y])

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(6700, )))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

epochs = 25
lr = 0.01
decay = lr / epochs
sgd = SGD(lr=lr, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x, y, batch_size=32, epochs=epochs)
model.save("model.h5")

In [ ]:
model.fit(x, y, batch_size=32, epochs=10)

In [ ]:
y_predicted = model.predict(x)

k = 0

for i, j in zip(y, y_predicted):
    if np.argmax(i) != np.argmax(j):
        print(k)
    k += 1

In [ ]:
model.save("model.h5")

In [ ]:
%%timeit
np.argmax(model.predict_on_batch(np.random.rand(1, 6700)))

In [ ]:
img = x[100].reshape(67, 100, 1)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import socket
import struct

model = load_model("model.h5")

UDP_IP = "127.0.0.1"
UDP_PORT_RECV = 9003
UDP_PORT_SEND = 9004


send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
recv.bind((UDP_IP, UDP_PORT_RECV))

send.sendto(struct.pack("i", 0), (UDP_IP, UDP_PORT_SEND))

while True:
    data, addr = recv.recvfrom(26800)
    inp = struct.unpack('6700f', data)
    inp = np.array(inp)
    inp = inp.reshape(1, 6700)
    ret = model.predict(inp)
    ret = np.argmax(ret[0])
    send.sendto(struct.pack("i", ret), (UDP_IP, UDP_PORT_SEND))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from collections import deque
import random

import socket
import struct


class Env:
    def __init__(self):
        self.UDP_IP = "127.0.0.1"
        self.UDP_PORT_RECV = 9003
        self.UDP_PORT_SEND = 9004

        self.send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.recv.bind((self.UDP_IP, self.UDP_PORT_RECV))

    def reset(self):
        self.send.sendto(struct.pack("i", -1), (self.UDP_IP, self.UDP_PORT_SEND))
        data, _ = self.recv.recvfrom(26800)
        data = np.array(struct.unpack('6700f', data)).reshape(1, 6700)
        return data

    def step(self, action):
        self.send.sendto(struct.pack("i", action), (self.UDP_IP, self.UDP_PORT_SEND))
        data, _ = self.recv.recvfrom(26800)
        data = np.array(struct.unpack('6700f', data)).reshape(1, 6700)


class Train:
    def __init__(self, env):
        self.env = env
        self.gamma = 0.99
        self.epsilon = 1
        self.epsilon_decay = 0.05
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.replay_buffer = deque(maxlen=20000)
        self.train_network = self.create_model()
        self.episode_num = 400
        self.iteration_num = 201
        self.num_pick_from_buffer = 32
        self.target_network = self.create_model()
        self.target_network.set_weights(self.train_network.get_weights())

    def create_model(self):
        model = Sequential()
        model.add(Dense(32, activation='relu', input_shape=(6700,)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(3, activation='softmax'))
        return model

    def get_best_action(self, state):
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.rand(1) < self.epsilon:
            action = np.random.randint(0, 3)
        else:
            action = np.argmax(self.train_network.predict(state)[0])
        return action

    def train_from_buffer(self):
        if len(self.replay_buffer) < self.num_pick_from_buffer:
            return
        samples = random.sample(self.replay_buffer, self.num_pick_from_buffer)
        states = []
        new_states = []
        for sample in samples:
            state, action, reward, new_state, done = sample
            states.append(state)
            new_states.append(new_state)

        new_array = np.array(states)
        states = new_array.reshape(self.num_pick_from_buffer, 2)
        new_array2 = np.array(new_states)
        new_states = new_array2.reshape(self.num_pick_from_buffer, 2)
        targets = self.train_network.predict(states)
        new_state_targets = self.target_network.predict(new_states)

        i = 0
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = targets[i]
            if done:
                target[action] = reward
            else:
                Q_future = max(new_state_targets[i])
                target[action] = reward + Q_future * self.gamma
            i += 1

        self.train_network.fit(states, targets, epochs=1, verbose=0)

    def original_try(self, current_state, eps):
        reward_sum = 0
        max_position = -99

        for i in range(self.iteration_num):
            best_action = self.get_best_action(current_state)

            new_state, reward = self.env.step(best_action)
            new_state = new_state.reshape(1, 2)
            if new_state[0][0] > max_position:
                max_position = new_state[0][0]
            if new_state[0][0] >= 0.5:
                reward += 10
            self.replay_buffer.append([current_state, best_action, reward, new_state])
            self.train_from_buffer()
            reward_sum += reward
            current_state = new_state

        if i >= 199:
            print("Failed to finish task in episode {}".format(eps))
        else:
            print("Success in episode {}, used {} iterations!".format(eps, i))
            self.train_network.save('./train_networkInEPS{}.h5'.format(eps))

        self.target_network.set_weights(self.train_network.get_weights())

        print("now epsilon is {}, the reward is {} maxPosition is {}".format(max(self.epsilon_min, self.epsilon),
                                                                             reward_sum, max_position))
        self.epsilon -= self.epsilon_decay

    def start(self):
        for eps in range(self.episode_num):
            current_state = self.env.reset()
            self.original_try(current_state, eps)


Train().start()